In [33]:
import torch

- https://github.com/mctosima/pengantarDLpytorch/blob/main/IPYNB%20Code/02_Autograd.ipynb

In [34]:
a = torch.ones(2, 2, requires_grad=True) # memungkinkan pytorch untuk menghitung gradient secara otomatis
print(a)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [35]:
x = torch.randn(3, requires_grad=True)
print(x)

tensor([ 0.6979, -2.2230,  0.3031], requires_grad=True)


In [36]:
y = x + 2
print(y)

tensor([ 2.6979, -0.2230,  2.3031], grad_fn=<AddBackward0>)


In [37]:
z = y * y * 3
print(z)

tensor([21.8356,  0.1492, 15.9123], grad_fn=<MulBackward0>)


In [38]:
z = z.mean()
print(z)

tensor(12.6324, grad_fn=<MeanBackward0>)


# Menghitung Gradien
Gradien dapat dihitung karena nilai dari z berupa skalar dan bukan vektor. Apabila misalnya kita hanya melakukan operasi z = y * y * 3, maka nilai dari z tidak berupa vektor dan tidak dapat dihitung gradiennya. Lalu bagaimana jika ingin melakukan backpropagation terhadap nilai yang berupa vektor?

In [39]:
# dz/dx
z.backward()
print(x.grad)

tensor([ 5.3958, -0.4460,  4.6061])


In [40]:
x = torch.randn(3, requires_grad=True)
y = x * 2
z = y * y * 3

# kita membutuhkan v sebagai array yang akan membantu melakukan backward terhadap z
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([-1.0366e+00, -1.7133e+01, -8.3580e-04])


# Menghentikan Tracking Gradien

In [41]:
x = torch.randn(3, requires_grad=True)
y = torch.randn(3, requires_grad=True)
z = torch.randn(3, requires_grad=True)
print(x)
print(y)
print(z)

tensor([ 0.4310, -0.8609, -0.9427], requires_grad=True)
tensor([-0.2413,  1.2600,  0.6018], requires_grad=True)
tensor([-0.2172, -0.5605,  1.2232], requires_grad=True)


# Cara Pertama

In [42]:
x.requires_grad_(False)
print(x)

tensor([ 0.4310, -0.8609, -0.9427])


# Cara Kedua

In [43]:
y = y.detach()
print(y)

tensor([-0.2413,  1.2600,  0.6018])


# Cara Ketiga

In [44]:
with torch.no_grad():
    z = x + y
    print(z)

tensor([ 0.1897,  0.3991, -0.3409])


# Gradien yang salah akibat dari backpropagation
Ketika kita melakukan backpropagation dengan epoch yang berjumlah lebih dari satu, maka kita akan mendapatkan nilai gradien yang salah seperti pada kode di bawah ini. Nilai dari gradien terakumulasi pada setiap perulangan epoch.

In [45]:
weights = torch.ones(4, requires_grad=True)
for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])


Untuk mengatasi kesalahan di atas, maka kode di bawah ini akan menghilangkan nilai gradien yang salah dengan cara mengosongkan nilai gradien

In [46]:
weights = torch.ones(4, requires_grad=True)
for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print(weights.grad)
    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


Hal yang sama juga perlu dilakukan ketika menggunakan optimizer

In [59]:
# from torch import optim
# weights = torch.ones(4, requires_grad=True)
# print(weights)
# optimizer = torch.optim.SGD(weights, lr=0.01)
# # optimizer.step()
# # gunakan optimizer untuk mengoptimalkan nilai weights
# # optimizer.zero_grad()